<a href="https://colab.research.google.com/github/NeuralClassifier/OutlierDetect/blob/main/HDBSCAN_OutlierLabel_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os as oo

In [ ]:
!unzip /content/reduced_synthetic_data_local_global.zip

In [ ]:
!unzip /content/outlierScore_perData_withNoise.zip

In [ ]:
backbone_path = '/content/reduced_synthetic_data_local_global/'
redc = [30,50,70,90]
over = [1,2,3,4]
dim = [2,10,20,40,80,100]
for r in redc:
  redc_path = backbone_path + str(r)+'p/outlierScore_perData/refined/'
  for o in over:
    over_path = redc_path + str(o)+'/'
    for d in dim:
      clear_output(wait=True)
      final_path = over_path+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_refined.xlsx'
      print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d)
      df = pd.read_excel(final_path)
      df.drop(['Unnamed: 0'],axis=1,inplace=True)
      #thres = int(input('Enter the threshold: '))
      min_pts = list(df.mpts.unique())
      for mp in min_pts:
        df_temp = df[df['mpts']==mp]
        os = np.array(df_temp['outlier score'])
        threshold = pd.Series(os).quantile(0.92)
        sns.distplot(os[np.isfinite(os)], rug=True)
        plt.show()
        lbs = []
        for i in range(len(df_temp)):
          if df_temp.iloc[i,-1] > threshold:
            lbs.append(1)
          else:
            lbs.append(0)
        lbs=pd.DataFrame(np.array(lbs),columns=['HDBSCAN_outliers'])
        df_temp=pd.concat([df_temp,lbs],axis=1)
        if mp == 2:
          df_final = df_temp
        else:
          df_final = pd.concat([df_final,df_temp],axis=0)
      df_final.to_excel('/content/HDBSCAN_outlier/'+str(r)+'p/'+str(o)+'/'+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'_hdbLabls.xlsx')


In [ ]:
!zip -r /content/HDBSCAN_outlier.zip /content/HDBSCAN_outlier

In [ ]:
!rm -rf /content/HDBSCAN_outlier

In [ ]:
def seg_RealData(df,lim):
  for i in range(lim+1):
    df_temp = df[df['data index']==i]
    if i ==0:
      df_final = df_temp 
    else:
      df_final = pd.concat([df_final,df_temp],axis=0)
  return df_final

backbone_path = '/content/outlierScore_perData_withNoise/'
redc = [70]
over = [4]
dim = [2,10,20,40,80,100]
noise = [20,30,40,50]

oo.mkdir('/content/HDBSCAN_outlier_noise/')

for r in redc:
  #redc_path = backbone_path + str(r)+'p/outlierScore_perData/refined/'
  for o in over:
    #over_path = redc_path + str(o)+'/'
    for d in dim:
      for n in noise:
        clear_output(wait=True)
        final_path = backbone_path+'OS_EachPt_mpts_od'+str(d)+'r'+str(r)+'n'+str(n)+'.xlsx'
        print('Reduction: ',r,', Overlap: ',o,', Dimension: ',d,', Noise: ',n)
        df = pd.read_excel(final_path)
        df.drop(['Unnamed: 0'],axis=1,inplace=True)
        df = seg_RealData(df,339)
        df.reset_index(drop=True, inplace=True)
        #thres = int(input('Enter the threshold: '))
        min_pts = list(df.mpts.unique())
        for mp in min_pts:
          df_temp = df[df['mpts']==mp]
          df_temp.reset_index(drop=True, inplace=True)
          print(df_temp)
          os = np.array(df_temp['outlier score'])
          threshold = pd.Series(os).quantile(0.92)
          #sns.distplot(os[np.isfinite(os)], rug=True)
          #plt.show()
          lbs = []
          for i in range(len(df_temp)):
            if df_temp.iloc[i,-1] > threshold:
              lbs.append(1)
            else:
              lbs.append(0)
          lbs=pd.DataFrame(np.array(lbs),columns=['HDBSCAN_outliers'])
          df_temp=pd.concat([df_temp,lbs],axis=1)
          if mp == 2:
            df_final = df_temp
          else:
            df_final = pd.concat([df_final,df_temp],axis=0)
            df_final.reset_index(drop=True, inplace=True)
          print(df_final)
        df_final.to_excel('/content/HDBSCAN_outlier_noise/'+'OS_EachPt_mpts_o'+str(o)+'d'+str(d)+'r'+str(r)+'n'+str(n)+'_hdbLabls.xlsx')


In [ ]:
!zip -r /content/HDBSCAN_outlier_noise.zip /content/HDBSCAN_outlier_noise

In [ ]:
df_final